In [ ]:
#Code to get the load_csv_database function.
#We pass this function as context so there's no need to include it when we parse the notebook
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..", "..")))
from spider2_utils import load_csv_database


-setup-

In [ ]:
import pandas as pd
_database = load_csv_database("WWE", rows_limit=-1)

Belts = _database["Belts"]
Cards = _database["Cards"]
Events = _database["Events"]
Locations = _database["Locations"]
Matches = _database["Matches"]
Promotions = _database["Promotions"]
Wrestlers = _database["Wrestlers"]

# Question
For the NXT title that had the shortest match (excluding titles with "title change"), what were the names of the two wrestlers involved?

# Step 1: Rename 'name' columns in each dataframe to avoid conflicts during merge

In [ ]:
Belts = Belts.rename(columns={'name': 'name_belt'})
Loser_Wrestlers = Wrestlers.rename(columns={'name': 'name_loser'})
Winner_Wrestlers = Wrestlers.rename(columns={'name': 'name_winner'})
Cards = Cards.rename(columns={'name': 'name_card'})
Locations = Locations.rename(columns={'name': 'name_location'})
Events = Events.rename(columns={'name': 'name_event'})
Promotions = Promotions.rename(columns={'name': 'name_promotion'})

# Step 2: Merge All Relevant Tables
Join the Belts, Matches, Wrestlers, Cards, Locations, Events, and Promotions tables to create a detailed match DataFrame.

In [ ]:
# Merge all relevant tables to replicate the SQL CTE MatchDetails
match_details = Matches.merge(Belts, left_on='title_id', right_on='id', suffixes=('', '_belt'))
match_details = match_details.merge(Winner_Wrestlers, left_on='winner_id', right_on='id', suffixes=('_winner', '_w1'))
match_details = match_details.merge(Loser_Wrestlers, left_on='loser_id', right_on='id', suffixes=('_loser', '_w2'))
match_details = match_details.merge(Cards, left_on='card_id', right_on='id', suffixes=('', '_card'))
match_details = match_details.merge(Locations, left_on='location_id', right_on='id', suffixes=('', '_loc'))
match_details = match_details.merge(Events, left_on='event_id', right_on='id', suffixes=('', '_event'))
match_details = match_details.merge(Promotions, left_on='promotion_id', right_on='id', suffixes=('', '_promo'))

# Step 3: Filter for NXT Promotion and Valid Titles/Duration
Filter for NXT promotion, non-empty duration, non-empty title, and exclude titles containing 'title change'.

In [ ]:
# Exclude titles with 'title change' in the name
exclude_titles = Belts[Belts['name_belt'].str.contains('title change', case=False, na=False)]['name_belt'].unique()
match_details = match_details[(match_details['name_promotion'] == 'NXT') &
                              (match_details['duration'] != '') &
                              (match_details['name_belt'] != '') &
                              (~match_details['name_belt'].isin(exclude_titles))]

# Step 4: Add Match Description and Rank by Duration
Create a 'matches' column with the format 'winner vs loser' and rank matches by duration within each title.

In [ ]:
match_details['matches'] = match_details['name_winner'] + ' vs ' + match_details['name_loser']
match_details['match_duration'] = match_details['duration']
match_details['titles'] = match_details['name_belt']
match_details['rank'] = match_details.groupby('titles')['match_duration'].rank(method='first', ascending=True)

# Step 5: Select the Shortest Match for Each Title
Filter to only the shortest match (rank 1) for each title.

In [ ]:
rank1 = match_details[match_details['rank'] == 1].copy()

# Step 6: Extract Wrestler Names from the Shortest Match
Split the 'matches' column to get the two wrestler names for the overall shortest match.

In [ ]:
shortest = rank1.sort_values('match_duration').iloc[0]
wrestler1 = shortest['matches'].split(' vs ')[0]
wrestler2 = shortest['matches'].split(' vs ')[1]
result = pd.DataFrame({'wrestler1': [wrestler1], 'wrestler2': [wrestler2]})
result

In [ ]:
# gold output
#wrestler1	wrestler2
#Bron Breakker	Duke Hudson